In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

In [14]:
def seed_everything(seed=42):
    # Setting random seed for various libraries
    os.environ['PYTHONHASHSEED'] = str(seed)
    #random.seed(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True

In [15]:
data_folder_path = "data\DataHourlyChina"
TARGET = "PM2.5"
RANDOM_SEED = 42
seed_everything(seed=RANDOM_SEED)

df = pd.read_csv(os.path.join(data_folder_path, "POST_EDA_POST_FEAT_ENG_STANDARDIZED.csv"), index_col="datetime")
df.head()

,year,PM10,SO2,NO2,CO,O3,TEMP,RAIN,WSPM,wd_ENE,...,cos_hour,month_sin,dayofweek_sin,dayofweek_cos,TEMP_x_CO,NO2_x_RAIN,WSPM_X_SO2,PM10_diff1,TEMP_diff1,PM2.5
datetime,,,,,,,,,,,,,,,,,,,,,
2013-03-01 01:00:00,-1.412466,-1.078452,-0.593658,-1.424646,-0.809090,0.374490,-1.288641,-0.073894,2.487723,-0.356598,...,1.366067,1.423946,-0.612319,-1.275284,-0.846123,-0.075799,-0.202244,0.110008,-0.303442,8.0
2013-03-01 02:00:00,-1.412466,-1.089070,-0.549144,-1.342647,-0.809090,0.304318,-1.288641,-0.073894,3.236048,-0.356598,...,1.224784,1.423946,-0.612319,-1.275284,-0.846123,-0.075799,0.037109,-0.027529,-0.000201,7.0
2013-03-01 03:00:00,-1.412466,-1.099689,-0.282056,-1.315314,-0.809090,0.286775,-1.314966,-0.073894,1.157367,-0.356598,...,1.000035,1.423946,-0.612319,-1.275284,-0.852138,-0.075799,0.195810,-0.027529,-0.227632,6.0
2013-03-01 04:00:00,-1.412466,-1.131543,-0.237542,-1.287982,-0.809090,0.286775,-1.367616,-0.073894,0.242748,-0.356598,...,0.707136,1.423946,-0.612319,-1.275284,-0.864169,-0.075799,-0.066958,-0.082543,-0.455063,3.0
2013-03-01 05:00:00,-1.412466,-1.110307,0.029546,-1.123985,-0.725066,0.181516,-1.385166,-0.073894,1.656251,-0.356598,...,0.366048,1.423946,-0.612319,-1.275284,-0.882884,-0.075799,1.041351,0.054994,-0.151822,5.0


## Checking Linear Assumptions

### Assumption 1: Random Sampling

#### Data must represent what we want to predict (Train: older data.  Test: newer data.)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(TARGET, axis=1), df[TARGET], test_size=0.2, random_state=RANDOM_SEED)
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

Train shape: (28045, 35)
Test shape: (7012, 35)


### Assumption 2: No Perfect Multicollinearity

#### It was handled in EDA.

## Assumption 3: Homoscedasticity

### The errors should be spread out evenly to make sure the model is stable.

In [17]:
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Fit the model
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()

# Run Breusch-Pagan test
bp_test = het_breuschpagan(model.resid, X_with_const)

# Extract results
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))

{'Lagrange multiplier statistic': 10004.030535653688, 'p-value': 0.0, 'f-value': 399.5547501153778, 'f p-value': 0.0}


### p-value < 0.05 → Heteroscedasticity exists → assumption is violated → linear regression is unreliable.
#### We may try to counter/level it by outlier removal (done in EDA) and log-transforming the target:

In [18]:
# select all positive features
non_negative_mask = df > 0
columns_with_no_negatives = non_negative_mask.all(axis=0)
log_transform = df.columns[columns_with_no_negatives]
log_transform

Index(['PM2.5'], dtype='object')

In [19]:
# transform the target
df[TARGET] = np.log(df[TARGET])

In [20]:
# Prepare data
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Fit the model
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()

# Run Breusch-Pagan test
bp_test = het_breuschpagan(model.resid, X_with_const)

# Extract results
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))


{'Lagrange multiplier statistic': 1392.1147369794294, 'p-value': 4.869912132913048e-270, 'f-value': 41.376942025455584, 'f p-value': 6.827399017149638e-276}


### Didn't help, the Linear Regression model is unreliable.

## Conclusion

### There is no need to check the remaining hypothesis, the Linear assumption is rejected.

### Other models to consider must take into account the fact of absense of linearity and presence of homoscedasticity. Possible candidates are tree-based models: Random Forest, XGBoost, LGBM, CatBoostRegressor, ExtraTreesRegressor.

In [21]:
df.to_csv(os.path.join(data_folder_path, "POST_EDA_POST_FEAT_ENG_STANDARDIZED_TARGET_TRANSFORMED.csv"), index=True, index_label='datetime')